In [14]:
import logging
import tensorflow as tf
import keras

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

tf.__version__

'2.4.1'

In [39]:
import tensorflow_datasets as tfds
from tensorflow.train import Feature, Example, Features, BytesList, Int64List
from pathlib import Path

def ch13_ex_9():
    val_fraction = 0.8
    seed = 101
    train_dir = 'data/train'
    val_dir = 'data/val'

    
    def create_dataset(dataset, data_dir):
        filepaths = []
        Path(data_dir).mkdir(parents=True, exist_ok=True)
        for i, d in enumerate(dataset):
            ser_image = tf.io.serialize_tensor(d['image']).numpy()
            image_example = Example(
                features=Features(
                    feature={
                        'image': Feature(bytes_list=BytesList(value=[ser_image])),
                        'label': Feature(int64_list=Int64List(value=[d['label']]))
                    }
                )
            )
            filepath = f'{data_dir}/image{i}.tfrecord'
            with tf.io.TFRecordWriter(filepath) as f:
                f.write(image_example.SerializeToString())
            filepaths.append(filepath)
        return filepaths
                
    def create_pipeline(filepaths):
#         def process_file(filepath):
#             parsed
        image_descr = {
            'image': tf.io.FixedLenFeature([], tf.int64, default_value=0),
            'label': tf.io.FixedLenFeature([], tf.int64, default_value=0)
        }
        for ser in tf.data.TFRecordDataset(filepaths):
            log.info(tf.io.parse_single_example(ser, image_descr))
            break
        dataset = tf.data.Dataset.list_files(filepaths, seed=seed).interleave(lambda path: tf.io.parse_single_example(path, image_descr), 
                                                                              num_parallel_calls=tf.data.AUTOTUNE)
        log.info(f'Dataset: {dataset}')
    
    dataset = tfds.load("fashion_mnist")
    log.info(f'Loaded dataset: {dataset}')
    train_idx = int(len(dataset['train']) * val_fraction)
    train = dataset['train'].take(train_idx).shuffle(101, seed=seed, reshuffle_each_iteration=True)
    log.info(f'Train dataset len: {len(train)}')
    val = dataset['train'].skip(train_idx)
    log.info(f'Val dataset len: {len(val)}')
#     create_dataset(train, data_dir=train_dir)
    val_filepaths = create_dataset(dataset=val, data_dir=val_dir)
    create_pipeline(val_filepaths)

             
ch13_ex_9()

2021-04-17 15:26:27,983 : INFO : Load dataset info from /Users/mkhokhlush/tensorflow_datasets/fashion_mnist/3.0.1
2021-04-17 15:26:27,985 : INFO : Reusing dataset fashion_mnist (/Users/mkhokhlush/tensorflow_datasets/fashion_mnist/3.0.1)
2021-04-17 15:26:27,985 : INFO : Constructing tf.data.Dataset for split None, from /Users/mkhokhlush/tensorflow_datasets/fashion_mnist/3.0.1
2021-04-17 15:26:28,041 : INFO : Loaded dataset: {'train': <PrefetchDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>, 'test': <PrefetchDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>}
2021-04-17 15:26:28,043 : INFO : Train dataset len: 48000
2021-04-17 15:26:28,043 : INFO : Val dataset len: 12000


InvalidArgumentError: Key: image.  Data types don't match. Data type: string but expected type: int64 [Op:ParseExampleV2]